<a href="https://colab.research.google.com/github/rdstew11/Chess/blob/working/Crate_and_Barrel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Crate and Barrel Project**

First thing to do is import pandas and take a look at our data. I downloaded the csv files directly from [Kaggle](https://www.kaggle.com/c/otto-group-product-classification-challenge/data?select=train.csv) and then uploaded it this Jupyter notebook.

## **Looking at the Data**

In [47]:
import pandas as pd;

train = pd.read_csv("train.csv")
train = train.set_index('id')
test = pd.read_csv("test.csv")
test = test.set_index('id')
test

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,3,0,0,0,3,2,1,0,0,0,0,0,0,0,5,3,1,1,0,0,0,0,0,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11,1,20,0,0,0,0,0
2,2,2,14,16,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,2,0,4,0,4,0,0,0,0,2,0,0,0,8,0,0,0,0,0,...,24,0,0,0,0,0,0,0,0,0,6,8,0,0,0,0,0,0,0,0,0,0,0,2,0,0,4,0,2,0,0,0,0,0,0,4,0,0,2,0
3,0,1,12,1,0,0,0,0,0,0,7,1,0,0,0,7,0,2,0,0,0,4,0,0,0,1,1,2,0,0,0,0,0,0,1,0,0,2,0,0,...,10,1,0,2,0,0,1,6,1,1,0,0,1,1,1,2,0,0,2,0,0,0,0,0,0,0,6,0,2,0,0,0,0,0,2,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,3,0,0,0,0,0,0,0,0,4,0,0,0,1,0,0,0,2,0,0,0,0,1,0,0,...,2,0,0,0,0,0,0,0,9,0,0,2,0,0,0,0,6,0,8,0,0,0,0,1,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0
5,1,0,0,1,0,0,1,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,4,0,5,16,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144364,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,5,0,0,1,0,1,0,2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,...,0,2,0,0,0,0,0,0,0,0,2,0,0,9,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,2,1,1,0,0,0,0,0
144365,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,1,11,0,0,0,0,0
144366,0,1,0,0,0,0,1,1,0,0,0,0,0,7,0,1,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,0,1,0,1,0,3,1,0,6,...,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,5,0,0,1,2,0,0,0,0,0,0,0,0,0,0,1,3,1,1,0,0,1,0,0


In [48]:
train

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,4,1,1,0,0,2,0,0,0,0,0,1,0,0,0,0,1,...,0,0,2,0,0,11,0,1,1,0,1,0,7,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,6,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
4,1,0,0,1,6,1,5,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,7,2,2,0,0,0,58,0,10,0,0,0,0,0,3,0,0,...,1,0,0,0,0,0,0,0,0,0,2,1,5,0,0,4,0,0,2,1,0,1,0,0,1,1,2,2,0,22,0,1,2,0,0,0,0,0,0,Class_1
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,4,0,1,0,0,0,1,0,0,0,0,1,0,0,0,Class_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61874,1,0,0,1,1,0,0,0,0,0,0,1,3,0,0,9,0,2,0,0,0,7,0,3,6,1,0,0,65,1,0,4,3,1,1,1,2,1,0,2,...,3,1,0,0,0,1,0,22,0,1,4,11,3,0,0,3,0,1,1,2,0,0,29,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,Class_9
61875,4,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,2,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0,0,1,5,0,0,0,0,0,0,0,0,0,1,0,11,0,0,0,0,0,0,2,0,0,2,0,0,1,0,Class_9
61876,0,0,0,0,0,0,0,3,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,2,0,0,0,0,...,0,0,0,0,19,0,0,4,0,0,0,0,18,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,Class_9


These are really big datasets with 93 features. I'm really excited to see how the various setups of XGBoost will perform.

The test DataFrame does not contain the target column because the accuracy will be determined through the Kaggle challenge. So the test DF doesn't need to be split any further. The train will have to be split into features and labels.

Looking at the data, all the values are integers with a wide range of values, so it will probably be beneficial to scale these values.


## **Data Preprocessing**

In [49]:
trainFeatures = train.drop('target', axis=1)
trainLabels = train['target']


trainFeatures

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,1,0,4,1,1,0,0,2,0,0,0,0,0,1,0,0,0,0,1,...,1,0,0,2,0,0,11,0,1,1,0,1,0,7,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,6,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,6,1,5,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,7,2,2,0,0,0,58,0,10,0,0,0,0,0,3,0,0,...,3,1,0,0,0,0,0,0,0,0,0,2,1,5,0,0,4,0,0,2,1,0,1,0,0,1,1,2,2,0,22,0,1,2,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,4,0,1,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61874,1,0,0,1,1,0,0,0,0,0,0,1,3,0,0,9,0,2,0,0,0,7,0,3,6,1,0,0,65,1,0,4,3,1,1,1,2,1,0,2,...,7,3,1,0,0,0,1,0,22,0,1,4,11,3,0,0,3,0,1,1,2,0,0,29,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0
61875,4,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,2,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2,0,0,1,5,0,0,0,0,0,0,0,0,0,1,0,11,0,0,0,0,0,0,2,0,0,2,0,0,1,0
61876,0,0,0,0,0,0,0,3,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,2,0,0,0,0,...,0,0,0,0,0,19,0,0,4,0,0,0,0,18,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0


In [53]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fColumns = list(trainFeatures.columns)
columns = list(test.columns)
test[fColumns] = scaler.fit_transform(test[fColumns])
trainFeatures[fColumns] = scaler.fit_transform(trainFeatures[fColumns])

test

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,...,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.050847,0.000000,0.000000,0.000000,0.088235,0.051282,0.021739,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.023474,0.100000,0.016129,0.014085,0.000000,0.000000,0.00000,0.00000,0.000000,0.03125,0.000000,0.000000,0.011628,0.00,0.023810,0.000000,0.023256,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.048387,0.00000,0.000000,0.000000,0.00000,0.0125,0.023256,0.0,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.150685,0.018519,0.540541,0.000000,0.0,0.000000,0.000000,0.000000
2,0.031250,0.044444,0.166667,0.195122,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.051282,0.043478,0.000000,0.000000,0.0,0.000000,0.000000,0.071429,0.000000,0.018779,0.000000,0.064516,0.000000,0.000000,0.000000,0.00000,0.03125,0.000000,0.00000,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.000000,...,0.279070,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.096774,0.12500,0.000000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.0000,0.066667,0.0,0.071429,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.064516,0.0,0.000000,0.090909,0.000000
3,0.000000,0.022222,0.142857,0.012195,0.0,0.0,0.000000,0.00,0.0,0.000000,0.189189,0.033333,0.000000,0.000000,0.000000,0.152174,0.000000,0.066667,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.016129,0.014085,0.083333,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.009615,0.000000,0.00,0.047619,0.000000,0.000000,...,0.116279,0.032258,0.0,0.052632,0.0,0.0,0.025641,0.162162,0.017857,0.018182,0.000000,0.00000,0.027778,0.008929,0.00578,0.0250,0.000000,0.0,0.057143,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.100000,0.0,0.071429,0.000000,0.0,0.000000,0.000000,0.000000,0.054054,0.000000,0.0,0.000000,0.000000,0.010989
4,0.000000,0.000000,0.000000,0.012195,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.538462,0.065217,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.014493,0.00000,0.00000,0.000000,0.06250,0.000000,0.000000,0.000000,0.00,0.023810,0.000000,0.000000,...,0.023256,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.160714,0.000000,0.000000,0.03125,0.000000,0.000000,0.00000,0.0000,0.139535,0.0,0.228571,0.000000,0.000000,0.000000,0.000000,0.035714,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.053571,0.013699,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,0.015625,0.000000,0.000000,0.012195,0.0,0.0,0.022727,0.02,0.0,0.050847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.018779,0.000000,0.000000,0.000000,0.000000,0.000000,0.02459,0.00000,0.007092,0.00000,0.011628,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.062500,0.00000,0.0000,0.000000,0.0,0.000000,0.011364,0.000000,0.057471,0.156863,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.010989,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.121622,0.000000,0.000000
...,...,...,...,...,...,...,...,...

In [54]:
from xgboost import XGBClassifier

params = {'n_estimators' : 600, 'tree method':'gpu_hist', 'predictor': 'gpu_predictor'}
model = XGBClassifier(**params)



Now it's important to think of what range of hyperparameters we should test. I have the model using 600 n_estimators, so I figured testing 100-300 n_estimators would be interesting. Followed by a max depth of 4-8. 

This may be asking too much for a decent performace, but we will see!



In [55]:
hyperparams = [{'n_estimators': [100,200,300], 'max_depth': [4,6,8]}]

In [57]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
param_comb = 5
folds=5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(model, param_distributions=hyperparams, n_iter=param_comb,  n_jobs=-1, 
                                   cv=skf.split(trainFeatures,trainLabels), verbose=3)

In [58]:
%%time 
result = random_search.fit(trainFeatures, trainLabels)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 77.3min finished


CPU times: user 7min 32s, sys: 1.55 s, total: 7min 34s
Wall time: 1h 24min 44s


Ooooh geez. Took 1h 24m 4s to train that, which doesn't including the 90+ mins I took trying to train is orginally with much more n_estimators. Gives a good perspective on how long somethings can take with that large of hyperparams!


I hope its a good one...

In [59]:
random_search.best_params_

{'max_depth': 8, 'n_estimators': 200}

Important to note that max_depth's best parameters were at the top limit of the range (8). This suggests that there is possibly a better selection for it.

In [68]:
predictions = random_search.best_estimator_.predict_proba(test)
predictions

array([[2.08597034e-04, 2.21764296e-01, 2.33052760e-01, ...,
        9.06281825e-03, 6.76281226e-04, 1.10148896e-04],
       [1.48107554e-03, 1.50842648e-02, 1.94536336e-03, ...,
        5.91539079e-03, 4.60445136e-01, 1.02361734e-03],
       [4.58626637e-05, 1.44139021e-05, 1.49426151e-05, ...,
        2.88766278e-05, 8.26078875e-04, 1.94872446e-05],
       ...,
       [1.47049828e-03, 5.76130688e-01, 2.70562023e-01, ...,
        2.94508482e-03, 2.38305031e-04, 5.73397556e-05],
       [1.24703205e-04, 1.98284283e-01, 2.78121419e-02, ...,
        6.11468975e-04, 3.43428692e-05, 3.61244711e-05],
       [1.72708227e-04, 4.60610181e-01, 3.83784980e-01, ...,
        1.26282379e-01, 2.97773804e-04, 2.59851775e-04]], dtype=float32)

In [69]:
from pandas import DataFrame
temp = DataFrame(predictions)
temp.to_csv('results.csv')